In [1]:
import pyspark
from pyspark.sql import SparkSession

In [20]:
pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()
    
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [21]:
def peek(mini_batch, batch_id):
    # first_row = mini_batch.take(1)
    first_row = mini_batch.limit(1).collect()
    print(first_row)
    if first_row:
        print(first_row[0])

In [22]:
query = green_stream.writeStream.foreachBatch(peek)

In [23]:
query.start()

24/04/09 01:28:11 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/03/1s0f2vgs2dz8zjydnjtvyf5h0000gn/T/temporary-665239c9-4776-4ac8-abf0-23766889ca7f. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/09 01:28:11 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


24/04/09 01:28:11 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


[]
